In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [4]:
df = pd.read_csv("../data/train_data.csv")
# removing duplicates
df = df.drop_duplicates()
# no. of nan values
df.isnull().sum().sum()

np.int64(0)

In [5]:
df.head()

,age,gender,income_bracket,loyalty_program,membership_years,churned,marital_status,number_of_children,education_level,occupation,...,distance_to_store,holiday_season,season,weekend,customer_support_calls,email_subscriptions,app_usage,website_visits,social_media_engagement,days_since_last_purchase
0,41,Female,Medium,No,3,No,Married,0,Bachelor's,Employed,...,72.83,No,Summer,No,18,No,Medium,2,Medium,6
1,40,Female,High,Yes,4,Yes,Single,4,PhD,Employed,...,44.60,Yes,Fall,No,16,Yes,Low,17,Medium,137
2,48,Male,High,No,4,Yes,Married,3,Bachelor's,Unemployed,...,46.15,Yes,Winter,Yes,11,Yes,High,68,High,275
3,19,Male,Low,No,9,Yes,Divorced,3,Bachelor's,Retired,...,0.62,No,Fall,No,15,Yes,Medium,61,Low,182
4,53,Female,Low,Yes,8,No,Divorced,2,Master's,Employed,...,38.09,Yes,Fall,Yes,0,Yes,High,19,High,330


In [6]:
# number of unique values for each column
nunique_counts = df.nunique()
cols_less_than = nunique_counts[nunique_counts == 2]
cols_less_than.count()

np.int64(6)

In [ ]:
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

def distance_correlation(x, y):
    x = np.atleast_2d(x).T
    y = np.atleast_2d(y).T
    a = squareform(pdist(x, metric='euclidean'))
    b = squareform(pdist(y, metric='euclidean'))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()
    dcov = np.sqrt(np.sum(A * B) / (x.shape[0] ** 2))
    dvar_x = np.sqrt(np.sum(A * A) / (x.shape[0] ** 2))
    dvar_y = np.sqrt(np.sum(B * B) / (x.shape[0] ** 2))
    return dcov / np.sqrt(dvar_x * dvar_y) if dvar_x > 0 and dvar_y > 0 else 0.0

def distance_correlation_matrix(df):
    # Only keep numeric columns
    df_numeric = df.select_dtypes(include=[np.number])
    cols = df_numeric.columns
    dist_corr_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    for c1 in cols:
        for c2 in cols:
            dist_corr_matrix.loc[c1, c2] = distance_correlation(df_numeric[c1], df_numeric[c2])
    return dist_corr_matrix

def mutual_information_scores(df):
    results = {}
    # Automatically convert categories to codes
    df_encoded = df.copy()
    for col in df_encoded.select_dtypes(include=["object", "category"]).columns:
        df_encoded[col] = df_encoded[col].astype("category").cat.codes
    
    for target in df_encoded.columns:
        features = df_encoded.drop(columns=[target])
        if df_encoded[target].nunique() > 10:
            mi = mutual_info_regression(features, df_encoded[target])
        else:
            mi = mutual_info_classif(features, df_encoded[target])
        results[target] = dict(zip(features.columns, mi))
    return pd.DataFrame(results)

# taking a small sample
sample_size = min(1000, len(df))
df_sample = df.sample(sample_size, random_state=42).reset_index(drop=True)

dist_corr_df = distance_correlation_matrix(df_sample)
mi_df = mutual_information_scores(df_sample)

print(f"Distance Correlation Matrix (numeric columns, {sample_size} samples):")
display(dist_corr_df)

print(f"Mutual Information Scores (numeric + encoded categories, {sample_size} samples):")
display(mi_df)

📏 Distance Correlation Matrix (numeric columns, 1000 samples):


,age,membership_years,number_of_children,transaction_id,product_id,quantity,unit_price,discount_applied,transaction_hour,week_of_year,...,product_return_rate,product_weight,product_shelf_life,promotion_id,customer_zip_code,store_zip_code,distance_to_store,customer_support_calls,website_visits,days_since_last_purchase
age,1.000000,0.053241,0.068168,0.049312,0.062906,0.053421,0.058681,0.063176,0.077977,0.049853,...,0.039615,0.052064,0.048369,0.049425,0.046126,0.065942,0.057465,0.037850,0.056771,0.031489
membership_years,0.053241,1.000000,0.041163,0.041247,0.036697,0.038347,0.045479,0.061309,0.045037,0.039485,...,0.039469,0.054290,0.070600,0.040253,0.054145,0.072813,0.050775,0.039323,0.034005,0.032477
number_of_children,0.068168,0.041163,1.000000,0.056198,0.056541,0.034457,0.062063,0.032533,0.034875,0.049820,...,0.041226,0.064246,0.045414,0.039260,0.039537,0.044009,0.033175,0.036976,0.056466,0.045853
transaction_id,0.049312,0.041247,0.056198,1.000000,0.037455,0.033873,0.044636,0.043162,0.035017,0.071271,...,0.062869,0.035841,0.039061,0.052060,0.041511,0.038535,0.064645,0.041974,0.032985,0.052026
product_id,0.062906,0.036697,0.056541,0.037455,1.000000,0.035743,0.043022,0.059028,0.042650,0.057560,...,0.043775,0.047607,0.041239,0.044788,0.062277,0.050391,0.060308,0.042295,0.033133,0.056672
quantity,0.053421,0.038347,0.034457,0.033873,0.035743,1.000000,0.065195,0.057963,0.052257,0.064089,...,0.037579,0.036595,0.054532,0.034297,0.043917,0.041115,0.064618,0.044773,0.043519,0.053152
unit_price,0.058681,0.045479,0.062063,0.044636,0.043022,0.065195,1.000000,0.046929,0.042835,0.042494,...,0.039469,0.042033,0.046411,0.045954,0.096196,0.075966,0.041889,0.046043,0.047509,0.044529
discount_applied,0.063176,0.061309,0.032533,0.043162,0.059028,0.057963,0.046929,1.000000,0.054693,0.037501,...,0.063332,0.039077,0.065613,0.052752,0.079843,0.046383,0.042640,0.048405,0.030401,0.054285
transaction_hour,0.077977,0.045037,0.034875,0.035017,0.042650,0.052257,0.042835,0.054693,1.000000,0.070917,...,0.039563,0.035538,0.055312,0.054857,0.043558,0.034451,0.044011,0.068402,0.045843,0.048428
week_of_year,0.049853,0.039485,0.049820,0.071271,0.057560,0.064089,0.042494,0.037501,0.070917,1.000000,...,0.059021,0.038713,0.066574,0.058402,0.035387,0.053557,0.032941,0.050525,0.032043,0.048938



📊 Mutual Information Scores (numeric + encoded categories, 1000 samples):


,age,gender,income_bracket,loyalty_program,membership_years,churned,marital_status,number_of_children,education_level,occupation,...,distance_to_store,holiday_season,season,weekend,customer_support_calls,email_subscriptions,app_usage,website_visits,social_media_engagement,days_since_last_purchase
gender,0.000000,NaN,0.012102,0.000000,0.000000,0.009508,0.000000,0.000000,0.009909,0.000000,...,0.000000,0.008978,0.000000,0.000000,0.000000,0.000000,0.000000,0.010067,0.000000,0.047949
income_bracket,0.000000,0.000000,NaN,0.003523,0.040743,0.009751,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.020759,0.008120,0.000000,0.005261,0.000000,0.000000
loyalty_program,0.000000,0.018448,0.000000,NaN,0.026401,0.000000,0.008455,0.000000,0.000000,0.026524,...,0.000000,0.000000,0.045387,0.000000,0.009027,0.000000,0.030481,0.000000,0.000000,0.000000
membership_years,0.024762,0.000000,0.000000,0.011080,NaN,0.000000,0.014865,0.005573,0.000000,0.000000,...,0.049118,0.000000,0.000000,0.003198,0.000000,0.012683,0.000000,0.005085,0.003526,0.000070
churned,0.000000,0.021895,0.001171,0.003221,0.036049,NaN,0.001494,0.010294,0.000000,0.000000,...,0.000000,0.000000,0.015541,0.015190,0.000000,0.000000,0.000000,0.002429,0.000000,0.007806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
app_usage,0.003827,0.019002,0.003303,0.000000,0.012556,0.000000,0.000000,0.000000,0.000000,0.022926,...,0.000000,0.000000,0.023936,0.000000,0.036476,0.009037,NaN,0.000000,0.000000,0.000000
website_visits,0.022484,0.010261,0.000000,0.000000,0.000000,0.000000,0.000000,0.021354,0.003566,0.035042,...,0.007213,0.000000,0.016696,0.000000,0.017697,0.000000,0.000000,NaN,0.008064,0.032839
social_media_engagement,0.037287,0.000000,0.040556,0.000000,0.003213,0.000000,0.009761,0.000000,0.034934,0.009164,...,0.022261,0.000000,0.000000,0.003358,0.004051,0.000000,0.000000,0.000000,NaN,0.000000
days_since_last_purchase,0.014850,0.046586,0.004479,0.000000,0.000000,0.001568,0.000000,0.026479,0.000000,0.004357,...,0.000000,0.003999,0.034006,0.026796,0.000000,0.027853,0.000000,0.036954,0.000000,NaN


In [23]:
# perform a correlation test between numerical columns and the target variable 'Price'
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('avg_purchase_value')
for i in numerical_cols:
    correlation = df[i].corr(df['avg_purchase_value'])
    # if abs(correlation) > 0.05:
    print(f"Correlation between {i} and avg_purchase_value: {correlation}")

Correlation between age and avg_purchase_value: 0.00022571172476673076
Correlation between membership_years and avg_purchase_value: -0.0017891850521989272
Correlation between number_of_children and avg_purchase_value: -0.0013622426741891778
Correlation between transaction_id and avg_purchase_value: 0.0017257036005454247
Correlation between product_id and avg_purchase_value: -1.8832782850825444e-05
Correlation between quantity and avg_purchase_value: 0.0022717571246670095
Correlation between unit_price and avg_purchase_value: -0.0008484805528193874
Correlation between discount_applied and avg_purchase_value: -0.0013247792335617083
Correlation between transaction_hour and avg_purchase_value: -0.0010808926027579522
Correlation between week_of_year and avg_purchase_value: 0.0008584929001343206
Correlation between month_of_year and avg_purchase_value: -0.0011885867745611305
Correlation between avg_discount_used and avg_purchase_value: -0.0024858504275598737
Correlation between online_purcha

In [24]:
# do chi squared test for correlation for categorical data
# from scipy.stats import chi2_contingency
# categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
# for q in [3, 4, 5]:
#     df[f'avg_purchase_value_bin_{q}'] = pd.qcut(df['avg_purchase_value'], q=q, duplicates='drop')
#     print(f"\n=== Using {q} quantile bins ===")
#     for col in categorical_cols:
#         contingency_table = pd.crosstab(df[col], df[f'avg_purchase_value_bin_{q}'])
#         if (contingency_table.values < 5).sum() > 0:
#             print(f"⚠️ Warning: some cells < 5 counts for {col}")
#         chi2, p, dof, expected = chi2_contingency(contingency_table)
#         print(f"{col}: p-value = {p:.4f}")


In [25]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
max_categories = 30  # tweak depending on dataset
filtered_categorical_cols = [col for col in categorical_cols if df[col].nunique() <= max_categories]
sample_df = df.sample(n=10000, random_state=42)


import statsmodels.api as sm
from statsmodels.formula.api import ols

alpha = 0.1
results = []

for col in filtered_categorical_cols:
    # Drop missing values only for the relevant columns
    sub_df = sample_df[[col, "avg_purchase_value"]].dropna()
    
    if sub_df[col].nunique() < 2:
        continue  # skip if column has only one unique category
    
    try:
        model = ols(f"avg_purchase_value ~ C({col})", data=sub_df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        p_value = anova_table["PR(>F)"][0]
        if p_value < alpha:
            results.append((col, p_value))
    except Exception as e:
        print(f"Skipping {col} due to error: {e}")

# Sort by p-value
results = sorted(results, key=lambda x: x[1])
# for col, p_value in results:
#     print(f"{col}: p-value = {p_value:.4f}")
results

/var/folders/l9/hd667pmd11g4b61wv2tpbsk00000gn/T/ipykernel_17995/1572633057.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_table["PR(>F)"][0]
/var/folders/l9/hd667pmd11g4b61wv2tpbsk00000gn/T/ipykernel_17995/1572633057.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_table["PR(>F)"][0]
/var/folders/l9/hd667pmd11g4b61wv2tpbsk00000gn/T/ipykernel_17995/1572633057.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, u

[('holiday_season', np.float64(0.0017703432772197018)),
 ('occupation', np.float64(0.018936171069983205)),
 ('promotion_type', np.float64(0.0397215853569443)),
 ('payment_method', np.float64(0.047947021353577354)),
 ('product_material', np.float64(0.08641196261656553))]

In [26]:
df = df.drop(columns=["loyalty_program", "churned", "education_level", "transaction_id", "transaction_date", "payment_method", "store_location", "transaction_hour", "day_of_week", "week_of_year", "month_of_year", "last_purchase_date", "preferred_store", "total_returned_items", "total_returned_value", "total_transactions", "product_rating", "product_review_count", "product_stock", "product_return_rate", "product_color", "product_manufacture_date", "product_expiry_date", "product_shelf_life", "promotion_type", "promotion_channel", "store_zip_code", "distance_to_store", "customer_support_calls"])

In [27]:
date_cols = []

# collect date columns
for i in df.columns:
    if 'date' in i.lower():
        date_cols.append(i)

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
    df[col] = df[col].astype('int64') / 1e9  # convert to float seconds
# drop all columns except 'holiday_season' 'occupation' 'promotion_type' 'payment_method' 'product_material'

df["avg_purchase_value"]

0         115.65
1         178.60
2         205.62
3          30.49
4         199.61
           ...  
799995    450.56
799996     33.87
799997    119.98
799998    292.99
799999    422.90
Name: avg_purchase_value, Length: 800000, dtype: float64

In [28]:
numeric_cols = df.select_dtypes(include=['number']).columns
# Outlier removal
def whisker(col):
    Q1,Q3 = np.percentile(col,[25,75])
    iqr = Q3 - Q1
    lw = Q1 - (1.5 * iqr)
    uw = Q3 + (1.5 * iqr)
    return lw, uw
# df.head()

In [29]:
for i in numeric_cols:
    lw,uw = whisker(df[i])
    df[i] = np.where(df[i]<lw, lw, df[i])
    df[i] = np.where(df[i]>uw, uw, df[i])
    # sns.boxplot(data=df, x=i)
    # plt.show()

In [30]:
# Data normalization
number = df.select_dtypes(include=np.number).columns.tolist()

for col in number:
    mean = df[col].mean()
    std = df[col].std()
    df[col] = (df[col] - mean) / std

In [33]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# One-hot encode categorical columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=False, dtype=float)
df["avg_purchase_value"]

0        -0.984196
1        -0.539157
2        -0.348133
3        -1.586255
4        -0.390622
            ...   
799995    1.383527
799996   -1.562359
799997   -0.953584
799998    0.269550
799999    1.187978
Name: avg_purchase_value, Length: 800000, dtype: float64

In [32]:
output_path = "/Users/avi/Desktop/university/sem_5/Machine_Learning/Avi_Dhall_A1/retail_task/data/cleaned_data.csv"
df.to_csv(output_path, index=False)